In [2]:
import numpy as np

leftList = np.load("/home/wngys/lab/DeepFold/protein_infor/leftIDArray.npy", allow_pickle=True).tolist()
toDeleList = np.load("/home/wngys/lab/DeepFold/protein_infor/negLess10IDArray.npy", allow_pickle=True).tolist()

toSelecList = list(set(leftList) - set(toDeleList))

In [4]:
import torch

toSelecNum = len(toSelecList) # 14018
perm = torch.randperm(toSelecNum)

# 训练集 验证集 测试集 比例 6:2:2
train_indices = perm[:int(toSelecNum*0.6)]   # 8410
valid_indices = perm[int(toSelecNum*0.6):int(toSelecNum*0.8)] # 2804
test_indices = perm[int(toSelecNum*0.8):toSelecNum] # 2804

toSelecTrainList = [toSelecList[i] for i in train_indices]
toSelecValidList = [toSelecList[i] for i in valid_indices]
toSelecTestList = [toSelecList[i] for i in test_indices]

np.save("/home/wngys/lab/DeepFold/pair/train.npy", toSelecTrainList)
np.save("/home/wngys/lab/DeepFold/pair/valid.npy", toSelecValidList)
np.save("/home/wngys/lab/DeepFold/pair/test.npy", toSelecTestList)



In [ ]:
toSelecTrainList = np.load("/home/wngys/lab/DeepFold/pair/train.npy", allow_pickle=True)
# valid 
# test

In [12]:
# 更改数据分布 确保每个batch至少一个正例
Dir = "/home/wngys/lab/DeepFold/pair/pair_bool_90/"
batch_size = 64
for id in toSelecTrainList:
    posi_data = []
    nega_data = []

    with open(Dir+id+".txt", "r") as f_r:
        lines = f_r.readline()
        if not lines:
            break
        id2 = lines.split('\t')[0]
        flag = lines.split('\t')[1].split("\n")[0]
        if flag == '1':
            posi_data.append((id2, flag))
        else:
            nega_data.append((id2, flag))
    
    posi_num = len(posi_data)
    nega_num = len(nega_num)
    total_num = posi_num + nega_num

    data_list = []

    if total_num > batch_size: # 小于就忽略
        batch_num = (total_num - 1)/ batch_size + 1
        left_num = total_num % batch_size # 没有用到?

        posi_num_batch = (posi_num - 1)/ batch_num + 1
        posi_left = posi_num % batch_num

        # 如果余数不为0 填充正例
        posi_data.extend(posi_data[:(batch_num-posi_left)])

        nega_cusor = 0
        for i in range(batch_num):
            data_list.extend(posi_data[i*posi_num_batch:(i+1)*posi_num_batch])
            if i < batch_num - 1:
                data_list.extend(nega_data[nega_cusor:nega_cusor+(batch_size-posi_num_batch)])
                nega_cusor += batch_size-posi_num_batch
            else: # 最后一个分支 负例也填不满batch_size
                data_list.extend(nega_data[nega_cusor:-1])

        # 将data_list 写入新文件夹 "/home/wngys/lab/DeepFold/pair/train_pair_bool_90"










    

['d4uhca1', 'd1w5sa1', 'd4kqia_', 'd4c08a_', 'd1bb9a_']
